In [154]:
import pandas as pd

In [155]:
API_KEY = "cpk_c3cffbe8058f42b0a706de0ca0a7c9db.c3d10ff88d155b95be180e2f2383ffe2.781qC6zcKy0gzECGtZYDalrMakmkZti8"

In [156]:
import time
import requests, json

def invoke_chutes_stream(prompt, model="openai/gpt-oss-120b-TEE", show_reasoning=False):
    url = "https://llm.chutes.ai/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json",
        "Accept": "text/event-stream",
    }

    body = {
        "model": model,
        "messages": [
            {"role": "system", "content": "Output ONLY the final answer text."},
            {"role": "user", "content": prompt},
        ],
        "stream": True,
        "max_tokens": 2048,
        "temperature": 0.0,
        "response_format": {
        "type": "json_schema",
            "json_schema": {
                "name": "answer_schema",
                "schema": {
                    "type": "object",
                    "properties": {
                        "answer": {
                            "type": "string",
                            "enum": ["A","B","C","D","E","F","G","H","I","J","K","L","M","N"],
                        }
                    },
                    "required": ["answer"],
                    "additionalProperties": False
                }
            }
        }
    }

    full = ""
    start = time.time()

    with requests.post(url, headers=headers, json=body, stream=True, timeout=120) as r:
        if not r.ok:
            print("STATUS:", r.status_code)
            print(r.text[:2000])
            r.raise_for_status()

        for raw in r.iter_lines(decode_unicode=True):
            if not raw or not raw.startswith("data:"):
                continue

            chunk = raw[len("data:"):].strip()
            if chunk == "[DONE]":
                break

            try:
                data = json.loads(chunk)
            except json.JSONDecodeError:
                continue

            choices = data.get("choices") or []
            if not choices:
                continue

            delta = (choices[0].get("delta") or {})

            # what you want (final)
            token = delta.get("content")

            # what you're actually receiving (reasoning stream)
            rtoken = delta.get("reasoning_content") or delta.get("reasoning")

            if token:
                #print(token, end="", flush=True)
                full += token
            elif show_reasoning and rtoken:
                # Only enable this if you WANT reasoning printed
                #print(rtoken, end="", flush=True)
                full += rtoken
    latency_s = time.time() - start

    obj = json.loads(full)
    return obj["answer"], obj, round(latency_s, 3)


In [163]:
import time
import random

results=[]
for i in range(709, len(prompts)):
    answer = invoke_chutes_stream(prompts.iloc[i,0], show_reasoning=False)
    print(i,answer)
    results.append((i, answer[0],answer[2]))
    time.sleep(random.uniform(1.5, 3.0))

709 ('G', {'answer': 'G'}, 11.137)
710 ('H', {'answer': 'H'}, 1.696)
711 ('E', {'answer': 'E'}, 1.653)
712 ('D', {'answer': 'D'}, 2.153)
713 ('D', {'answer': 'D'}, 1.261)
714 ('F', {'answer': 'F'}, 2.07)
715 ('B', {'answer': 'B'}, 2.002)
716 ('E', {'answer': 'E'}, 2.212)
717 ('F', {'answer': 'F'}, 1.722)
718 ('D', {'answer': 'D'}, 2.043)
719 ('C', {'answer': 'C'}, 2.279)
720 ('C', {'answer': 'C'}, 2.001)
721 ('F', {'answer': 'F'}, 1.47)
722 ('E', {'answer': 'E'}, 1.272)
723 ('D', {'answer': 'D'}, 2.458)
724 ('B', {'answer': 'B'}, 2.255)
725 ('D', {'answer': 'D'}, 2.292)
726 ('G', {'answer': 'G'}, 2.651)
727 ('E', {'answer': 'E'}, 2.774)
728 ('D', {'answer': 'D'}, 1.959)
729 ('F', {'answer': 'F'}, 1.693)
730 ('B', {'answer': 'B'}, 1.689)
731 ('E', {'answer': 'E'}, 2.191)
732 ('B', {'answer': 'B'}, 2.05)
733 ('B', {'answer': 'B'}, 2.966)
734 ('E', {'answer': 'E'}, 2.641)
735 ('F', {'answer': 'F'}, 1.734)
736 ('G', {'answer': 'G'}, 2.331)
737 ('E', {'answer': 'E'}, 1.811)
738 ('E', {'answ

In [164]:
print(len(results))

108


In [165]:
results[::-1]

[(816, 'D', 2.019),
 (815, 'B', 2.087),
 (814, 'F', 1.586),
 (813, 'C', 2.508),
 (812, 'C', 3.165),
 (811, 'B', 2.22),
 (810, 'D', 1.555),
 (809, 'D', 1.798),
 (808, 'G', 1.784),
 (807, 'D', 3.243),
 (806, 'E', 2.195),
 (805, 'F', 2.614),
 (804, 'F', 1.931),
 (803, 'B', 2.834),
 (802, 'D', 1.837),
 (801, 'B', 2.546),
 (800, 'D', 1.884),
 (799, 'F', 2.369),
 (798, 'E', 3.306),
 (797, 'C', 4.415),
 (796, 'E', 1.568),
 (795, 'D', 2.846),
 (794, 'D', 1.998),
 (793, 'E', 2.901),
 (792, 'F', 2.092),
 (791, 'B', 2.791),
 (790, 'D', 1.677),
 (789, 'B', 2.362),
 (788, 'D', 2.431),
 (787, 'D', 2.383),
 (786, 'D', 3.161),
 (785, 'G', 2.803),
 (784, 'B', 3.696),
 (783, 'E', 3.26),
 (782, 'B', 3.228),
 (781, 'C', 2.421),
 (780, 'C', 2.91),
 (779, 'C', 1.86),
 (778, 'E', 1.402),
 (777, 'C', 1.842),
 (776, 'E', 1.991),
 (775, 'C', 2.417),
 (774, 'B', 2.293),
 (773, 'G', 2.676),
 (772, 'F', 1.964),
 (771, 'C', 3.208),
 (770, 'F', 2.736),
 (769, 'E', 1.401),
 (768, 'F', 2.419),
 (767, 'E', 4.351),
 (76

In [166]:
import pandas as pd

df_results = pd.DataFrame(
    results,
    columns=["question_num", "pred_letter", "latency_s"]
)

df_results


,question_num,pred_letter,latency_s
0,709,G,11.137
1,710,H,1.696
2,711,E,1.653
3,712,D,2.153
4,713,D,1.261
...,...,...,...
103,812,C,3.165
104,813,C,2.508
105,814,F,1.586
106,815,B,2.087


In [167]:
df_results.to_csv('Old project/gpt-oss-709-816.csv', index=False)